#  LangChain의 개념과 주요 컴포넌트 이해

---

## LangChain이란 

핵심 내용:
- **LangChain**은 LLM 기반 애플리케이션 개발을 위한 프레임워크

- **Chain**은 작업을 순차적으로 실행하는 파이프라인 구조를 제공

- **Agent**는 자율적 의사결정이 가능한 실행 단위

결론:
- LangChain은 Chain과 Agent라는 두 가지 핵심 기능을 통해 LLM 애플리케이션 개발을 효율적으로 지원


    <div style="text-align: center;">
        <img src="https://python.langchain.com/svg/langchain_stack_112024_dark.svg" 
            alt="langchain_stack" 
            width="600" 
            style="border: 0;">
    </div>

## LangChain 컴포넌트 

- LangChain **주요 컴포넌트**: LLM/ChatModel, Prompt, Memory, Tool, Document Loader, Text Splitter, Embedding, Vectorstore

- **언어 처리 기능**은 LLM/ChatModel이 중심이 되며, Prompt와 Memory로 대화를 관리

- **문서 처리와 검색**은 Document Loader, Text Splitter, Embedding, Vectorstore가 담당

- **모듈성**이 핵심 특징으로, 독립적인 컴포넌트들을 조합해 RAG와 같은 복잡한 시스템을 구현 가능 

---

# 환경 설정 및 준비

In [ ]:
# 환경 변수 로드
from dotenv import load_dotenv
load_dotenv()

# 1. 모델 (Models)
- LLM, ChatModel 등으로 구분
- OpenAI, Anthropic, Google 등 다양한 모델을 지원
- 텍스트 생성, 대화, 요약 등의 작업을 수행

In [ ]:
from langchain_openai import ChatOpenAI

# OpenAI 모델을 사용하여 대화 생성
model = ChatOpenAI(model="gpt-4.1-mini")

# 모델에 메시지를 보내고 응답을 받기
response = model.invoke("안녕하세요!")

In [ ]:
# 응답 객체(AIMessage): 메시지(content)와 메타데이터(response_metadata 등)를 포함
response

In [ ]:
# 응답 객체의 메시지 내용 출력
print("답변: ", response.content)

In [ ]:
# 응답 객체의 메타데이터 출력
print("메타데이터: ", response.response_metadata)

# 2. 메시지 (Messages)
- Chat Model에서 사용할 수 있는 통합된 메시지 형식을 제공
- 각 모델 제공자의 특정 메시지 형식을 신경 쓰지 않고도 다양한 채팅 모델을 활용 가능

`1. HumanMessage`
- 사용자 역할에 해당 (user, human 등)
- 사용자의 입력을 처리

In [ ]:
from langchain_core.messages import HumanMessage

# 사용자 메시지 생성
human_message = HumanMessage(content="Glory를 한국어로 번역해주세요.")

# 번역 요청 및 응답 받기
response = model.invoke([human_message]).  # 메시지 리스트로 전달

# 답변 출력
print("답변: ", response.content)

In [ ]:
# 문자열을 입력하면, 자동으로 HumanMessage로 변환하여 요청
model.invoke("Glory를 한국어로 번역해주세요.")

`2. AIMessage`
- AI 모델의 응답을 표현


In [ ]:
# AI 모델의 응답 객체를 출력 
response

In [ ]:
# 모델 응답 텍스트 부분을 출력
response.content

In [ ]:
# 토큰 사용량 출력
response.usage_metadata

`3. SystemMessage`
- 시스템 역할에 해당 (system, developer 등)
- AI 모델의 동작과 제약사항을 정의하는데 사용


In [ ]:
from langchain_core.messages import SystemMessage 

# 시스템 메시지 생성
system_msg = SystemMessage(content="당신은 영어를 한국어로 번역하는 AI 어시스턴트입니다.")

# 메시지 객체 확인
system_msg

In [ ]:
# 번역 요청 (HumanMessage)과 시스템 메시지(SystemMessage)를 함께 사용
human_message = HumanMessage(content="Glory")
messages = [system_msg, human_message]

# 모델에 메시지를 보내고 응답 받기
response = model.invoke(messages)

# 답변 출력
print("답변: ", response.content)

# 3. 프롬프트 템플릿 (Prompt Template)
- 프롬프트 템플릿을 통해 일관된 입력 형식을 제공
    1. 사용자의 입력과 파라미터를 언어 모델이 이해할 수 있는 형태로 변환하는 도구
    2. 언어 모델에게 전달할 지시문을 만드는 틀
- 변수를 포함한 동적 프롬프트 생성이 가능
    1. 모든 템플릿은 딕셔너리 형태의 입력을 받아서 처리
    2. 출력은 PromptValue 형태로 반환되며, 이는 문자열이나 메시지 리스트로 변환 가능

`1. 문자열 프롬프트 템플릿 (String PromptTemplate)`
- 가장 기본적인 형태
- 단일 문자열을 형식화하는데 사용

In [ ]:
from langchain_core.prompts import PromptTemplate

# 템플릿 생성 
# "{topic}에 대한 이야기를 해줘"라는 템플릿을 사용하여
# topic이라는 변수를 포함하는 프롬프트를 생성
template = PromptTemplate.from_template("{topic}에 대한 이야기를 해줘")

# 템플릿 사용
# "고양이"라는 주제를 사용하여 프롬프트 생성
# invoke 메서드를 통해 템플릿에 값을 전달
prompt = template.invoke({"topic": "고양이"})

# 템플릿 출력
prompt

`2. 채팅 프롬프트 템플릿 (ChatPromptTemplate)`
- 여러 메시지를 포함하는 대화형 템플릿을 만들 때 사용

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# 채팅 템플릿 생성
# 여기서는 시스템 메시지와 사용자 메시지를 포함하여 정의
template = ChatPromptTemplate.from_messages([
    ("system", "당신은 도움이 되는 비서입니다"),
    ("user", "{subject}에 대해 설명해주세요")
])

# 템플릿 사용
prompt = template.invoke({"subject": "인공지능"})

# 출력
prompt

`3. 메시지 플레이스홀더 (MessagesPlaceholder)`
- 기존 메시지 목록을 템플릿의 특정 위치에 삽입할 때 사용

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

# 메시지 플레이스홀더가 있는 템플릿
template = ChatPromptTemplate.from_messages([
    ("system", "당신은 도움이 되는 비서입니다"),
    MessagesPlaceholder("chat_history")   # 채팅 기록을 플레이스홀더로 사용 (예: 이전 대화 내용) -> 이 위치에 메시지 목록을 추가할 수 있음
])

# 템플릿 사용
prompt = template.invoke({
    "chat_history": [
        HumanMessage(content="안녕하세요! 제 이름은 스티브입니다."),
        AIMessage(content="안녕하세요! 무엇을 도와드릴까요?"),
        HumanMessage(content="제 이름을 기억하나요?")
        ]
})

# 출력
prompt.messages

# 4. 출력 파서 (Output Parser)
1. **역할과 기능**
    - 모델의 텍스트 출력을 구조화된 데이터로 변환
    - 채팅 모델과 LLM의 출력을 정규화
    - 다운스트림 작업을 위한 데이터 형식 변환

2. **사용 시 고려사항**
    - OpenAI function calling과 같은 기능이 있는 경우, 해당 기능을 우선 사용

`(1) StrOutputParser`

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 기본적인 문자열 파서 사용
parser = StrOutputParser()

# 프롬프트 템플릿 설정
prompt = PromptTemplate.from_template("도시 {city}의 특징을 알려주세요")

# 모델 정의
model = ChatOpenAI(model='gpt-4.1-mini')

# 체인 구성
chain = prompt | model | parser

# 체인 실행
result = chain.invoke({"city": "서울"})

# 결과 출력
print(result)

`(2) 구조화된 출력 (with_structured_output 메소드)`

In [ ]:
from pydantic import BaseModel, Field

# Pydantic 클래스로 출력 구조를 정의
class CityInfo(BaseModel):
    name: str = Field(description="도시 이름")
    description: str = Field(description="도시의 특징")

In [ ]:
from pydantic import BaseModel, Field
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 1. 출력 스키마 정의
class CityInfo(BaseModel):
    name: str = Field(description="도시 이름")
    description: str = Field(description="도시의 특징")

# 2. 프롬프트 템플릿 생성
prompt = PromptTemplate.from_template("도시 {city}의 특징을 알려주세요.")

# 3. 모델 생성 및 구조화된 출력 바인딩
model = ChatOpenAI(model="gpt-4.1-mini", temperature=0)
structured_model = model.with_structured_output(CityInfo)

# 4. 프롬프트와 모델 체인 연결
chain = prompt | structured_model

# 5. 체인 실행
result = chain.invoke({"city": "서울"})

# 6. 결과 출력 (CityInfo 객체)
print(result)
print("-" * 20)
print(f"도시 이름: {result.name}")
print(f"특징: {result.description}")

# 5. 메모리 (Memory)
- 대화 기록을 저장하고 관리
- 컨텍스트 유지를 위한 다양한 메모리 타입을 제공
- 대화 요약, 버퍼링 등의 기능을 포함

In [ ]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage
from pydantic import BaseModel, Field
from typing import List

# 메모리 기반 히스토리 구현
class InMemoryHistory(BaseChatMessageHistory, BaseModel):
    """메모리 기반 채팅 메시지 히스토리"""

    # 메시지 목록
    messages: List[BaseMessage] = Field(default_factory=list)
    
    # 메시지 추가
    def add_messages(self, messages: List[BaseMessage]) -> None:
        self.messages.extend(messages)
    
    # 메시지 목록 초기화 
    def clear(self) -> None:
        self.messages = []

# 세션 저장소
store = {}

# 세션 ID로 히스토리 가져오기
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    """세션 ID에 해당하는 채팅 메시지 히스토리를 반환합니다."""

    # 세션 ID가 저장소에 없으면 새 InMemoryHistory 객체 생성
    if session_id not in store:
        store[session_id] = InMemoryHistory()
        
    # 해당 세션 ID의 히스토리 반환
    return store[session_id]

In [ ]:
# 채팅 모델과 프롬프트 설정
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory

prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 {subject}에 능숙한 비서입니다"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}")
])

chain = prompt | ChatOpenAI(model='gpt-4.1-mini')

# 히스토리 관리 추가
chain_with_history = RunnableWithMessageHistory(
    chain,    # 실행 체인
    get_session_history,  # 세션 ID에 해당하는 히스토리 가져오는 함수
    input_messages_key="question",     # 입력 메시지 키
    history_messages_key="history"     # 히스토리 메시지 키
)

In [ ]:
# 체인 실행
response = chain_with_history.invoke(
    {"subject": "수학", "question": "1+2는 얼마인가요?"},
    config={"configurable": {"session_id": "user1"}}
)

# 결과 출력
print(response)

In [ ]:
# 세션 ID로 히스토리 가져오기
get_session_history("user1").messages

In [ ]:
# 히스토리 이용해서 대화 진행
response = chain_with_history.invoke(
    {"subject": "수학", "question": "여기에 숫자 2를 곱하면 얼마인가요?"},
    config={"configurable": {"session_id": "user1"}}
)

# 결과 출력
print(response)

In [ ]:
# 세션 ID로 히스토리 가져오기
get_session_history("user1").messages

# 6. 에이전트 (Agent)
- 자율적 의사결정이 가능한 실행 단위
- LangChain에서는 Agent 클래스를 통해 에이전트 구현

In [ ]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import tool

# 프롬프트 템플릿 생성 - ReAct 에이전트에 필요한 변수들 포함
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 친절한 수학 선생님입니다."),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")    # 에이전트가 도구 호출 결과를 기록할 플레이스홀더
])

# 도구 정의
@tool
def add(a: float, b: float) -> float:
    """두 숫자를 더하는 도구"""
    return a + b

@tool
def subtract(a: float, b: float) -> float:
    """두 숫자를 빼는 도구"""
    return a - b

# 도구 목록 생성
tools = [
    add,
    subtract
]

# 에이전트 생성 (도구 호출)
agent = create_tool_calling_agent(
    llm=ChatOpenAI(model='gpt-4.1-mini'),
    tools=tools,
    prompt=prompt
)

# 에이전트 실행 도구 정의
agent_executor = AgentExecutor(
    agent=agent,      # 도구 호출 에이전트
    tools=tools,      # 도구 목록
    verbose=True,     # 상세 로그 출력
    )

# 에이전트 실행
agent_executor.invoke({"input": "100과 200을 더하면 얼마인가요?"})

---
## [실습]

### 1. 다음 조건을 만족하는 새로운 ChatPromptTemplate을 만드세요.
   - 시스템 메시지: "당신은 친절한 과학 선생님입니다"
   - 대화 기록을 포함
   - 사용자 질문을 받을 수 있는 형식


In [ ]:
# 여기에 코드를 작성하세요.

### 2. StrOutputParser를 사용하여 다음을 구현해보세요.
   - 앞의 프롬프트 템플릿을 사용하여 체인 구성 및 실행
   - "섭씨온도와 화씨온도 관계"를 설명해 달라고 요청하는 프롬프트 작성 
   - 결과 출력

In [ ]:
# 여기에 코드를 작성하세요.